In [1]:
# penguin_nn_logistic_equivalent.py
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the penguins dataset
penguins = sns.load_dataset('penguins')
# Drop rows with missing values
penguins.dropna(inplace=True)
# Features
#X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
X = penguins[['bill_length_mm', 'bill_depth_mm']]
# Target: 1 if Adelie, 0 otherwise
y = (penguins['species'] == 'Adelie').astype(int)

In [3]:
#scaler = StandardScaler()
#X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X= X / X.max()

In [4]:
lambda_keras = 10  # for C=5

model = tf.keras.Sequential([
    tf.keras.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(1,
        activation='sigmoid',
        use_bias=False
        #kernel_regularizer=regularizers.l2(lambda_keras)  # equivalent to sklearn's C=5
    )
])


In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [6]:
# Compile model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
history = model.fit(X, y, epochs=1000, verbose=1)

Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 0.6858  
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 0.6819 
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.6787 
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.6756 
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.6727 
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.6699 
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.6671 
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 0.6648 
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 0.6617 
Epoch 10/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 0.6595 
Epoch 11/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 0.6564 
Epoch 12/1000
11/11 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
train_acc_tf = history.history["accuracy"][-1]
print(train_acc_tf)

In [ ]:
x_min=0.5
x_max=1 #= X['bill_length_mm'].min(), X['bill_length_mm'].max()
y_min=0.5 
y_max=1 #= X['bill_depth_mm'].min(), X['bill_depth_mm'].max()

xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))

w = model.layers[0].get_weights()[0]
slope = -w[0] / w[1]
x_vals = np.array([x_min, x_max])
y_vals = slope * x_vals
slope = (-w[0] / w[1]).item()

# Predict probabilities across the grid
grid = np.c_[xx.ravel(), yy.ravel()]
Z = model.predict(grid)
Z = Z.reshape(xx.shape)

# --- Plot ---
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, levels=1, cmap='RdBu', alpha=0.6)
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)

# Add scatter plot of the data
y = pd.Series(y, index=X.index)  # ensure same index
plt.scatter(X.loc[y==1, 'bill_length_mm'], X.loc[y==1, 'bill_depth_mm'],
            c='blue', edgecolor='k', label='Adelie')
plt.scatter(X.loc[y==0, 'bill_length_mm'], X.loc[y==0, 'bill_depth_mm'],
            c='red', edgecolor='k', label='Other')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

plt.plot(x_vals, y_vals, 'k-', label=f"Decision line: depth = {slope:.2f} * length")
plt.xlabel('Bill length (standardized)')
plt.ylabel('Bill depth (standardized)')
plt.title('Decision Boundary (Adelie vs Others)')
plt.legend()
plt.show()